In [ ]:
# !pip install seaborn
# !pip install dask
# !pip install dask[dataframe]
# !pip install graphviz
# !pip install tqdm-joblib
# #!pip show "dask[dataframe]"
#!pip install librosa
#!pip install zarr dask numcodecs
#!pip show numcodecs zarr

#!pip install duckdb

In [ ]:
import pandas as pd
import multiprocessing

from library import ExperimentConfig
from library import GlobalVars
import library as lib
import data_library as data_lib
import sqlite_library as sql_lib

import helpers.dask_helper as dask_helper

# use this only at the beginning

"""
STEP 1 - Synchronize metadata with available audio files.

This function processes the original metadata CSV and prepares it for later use.
It performs the following operations:

1.1
- Eliminates unnecessary columns ('lat', 'long', 'rain', 'gust speed', 'weatherID', 'time')
- Creates a `file_name_prefix` column used to match each row with its corresponding audio files
- Adds a column indicating whether the corresponding audio file exists on disk
- Generates label-related columns with descriptive text (e.g., 'queen present', 'queen not present', etc.)
- Adds extra time-based columns derived from the datetime field (e.g., day, hour)

1.2
- Creates a new, cleaned and synchronized CSV that matches existing audio files
- Adds a `train1` column used to pre-define the train/test split

Parameters:
- `save_processed`: if True, the cleaned DataFrame will be saved to disk
- `show_df_stats`: if True, prints summary statistics about the metadata

Returns:
- A cleaned and synchronized DataFrame (`synch_df`)
"""
synch_df = lib.sync_metadata_with_audio(save_processed=True, show_df_stats=False)
print("Init done!")




In [ ]:
#features = ['mel', 'mfcc_13', 'delta_13', 'delta2_13', 'mfcc_20', 'mfcc_30', 'mfcc_40', 'mfcc_50', 'zcr', 'rms']
#features = ['mel', 'mfcc_13', 'delta_13', 'delta2_13', 'mfcc_20', 'delta_20', 'delta2_20',  'mfcc_30', 'delta_30', 'delta2_30', 'mfcc_40', 'delta_40', 'delta2_40', 'mfcc_50', 'delta_50', 'delta2_50', 'zcr', 'rms']

# """
# set segmentation parameters and features that will be extracted from audio files
# features - list of features that will be extracted, ex: features = ['mel', 'mfcc_13', 'delta_13', 'delta2_13', 'mfcc_20', 'mfcc_30', 'mfcc_40', 'mfcc_50', 'zcr', 'rms']
# """
features = ['pe-mfcc_40']
GlobalVars.set_segment_lenght_and_overlap(10,5)

# """
# STEP2
# Each sound file will be split in more subsegments (with or without overlapping)
# 2.1 
# - create new csv file fith list of splitted subsegments for speficied parameters
# """
df=pd.read_csv(GlobalVars.csv_data_sync_path )
lib.generate_segmentation_metadata(df, segment_length=GlobalVars.segment_length, overlap=GlobalVars.overlap, recreate_if_exists=False)

In [ ]:

# manager = multiprocessing.Manager()
# # variable  shared between processes

# global_config = manager.dict(
#     {"segment_lenght": GlobalVars.segment_length,  "overlap": GlobalVars.overlap, "features": features}
# )  
# print(global_config)
# """
# Extract defined feature for specified segmentation
# """
# datasetpath = GlobalVars.extended_dataset_file_path()
# dask_helper.process_csv(
#     datasetpath,
#     one_row_processor=None,
#     partition_processor=lambda batch, partition_info: sql_lib.extract_features_from_batch(df_partition=batch, partition_info=partition_info, global_config=global_config),
#     blocksize='500KB'
# )

# sql_lib.merge_all_partitions(features)

In [ ]:
"""
Test caching for one specified configuration
"""

cache_config_dict = {
    "segment_lenght": 10,            
    "segment_overlap": 5,         
    "feature": 'pe-mfcc_40',
    "vector_operation": 'mean_iqr25'
        
        
}
cache_cfg = ExperimentConfig(cache_config_dict)
data_lib.cache_one_configuration(cache_cfg)

In [ ]:
"""
Create caches for all configurations 
"""
data_lib.combine_all_data_parameters()
data_lib.cache_all_collections()